In [ ]:
# Code Javascript

In [ ]:
import dask.dataframe as dd
import os
import pandas as pd
import numpy as np
from dask.distributed import Client
import geopandas as gpd
import matplotlib.pyplot as plt
import hvplot.pandas
import sys
import xarray as xr
import warnings
from IPython.display import Markdown as md
import cartopy.crs as ccrs
import hvplot
import holoviews as hv
from geoviews import opts
import geoviews as gv
import geoviews.feature as gf
import s3fs
import fsspec
from bokeh.layouts import widgetbox
from bokeh.layouts import column as bokehCol
from bokeh.models.layouts import Column
from bokeh.models.widgets import DatePicker
from datetime import date
import panel as pn
import hvplot.xarray
from shapely.geometry import Polygon
from shapely.geometry import Point
import random
from geoviews import opts

pd.set_option('display.max_rows', 2000)
warnings.filterwarnings("ignore")

In [ ]:
# Dask client to implement distributed calculations (multicore or multinode if a cluster is available)
client = Client(silence_logs=40)

# 1. Introduction

Texte

# 2. Méthodologie

## 2.1 Domaine à l'étude

In [ ]:
bucket = 's3://era5-atlantic-northeast/zarr/land/reanalysis'

storage_options = {'endpoint_url': 'https://s3.us-east-2.wasabisys.com'}

ds = xr.open_zarr(fsspec.get_mapper(bucket,
                                    client_kwargs=storage_options,
                                    anon=True),
                  consolidated=True)

In [ ]:
ds

In [ ]:
bv_polygon_url = 'https://s3.us-east-2.wasabisys.com/watersheds-polygons/MELCC/json/040204/040204.json'

extent = Polygon([[-96, 63], [-96, 40], [-52, 40], [-52, 63]])

basin_polygon = gpd.read_file(bv_polygon_url).iloc[0].geometry
bounds_polygon = basin_polygon.bounds

# 1 3
# 0 1
# 2 1
# 2 3

gdf = gpd.GeoDataFrame(index=[0, 1, 3], 
                       crs={'init': 'epsg:4326'}, 
                       geometry=[extent, 
                                 basin_polygon,
                                  Polygon([[np.floor(5*bounds_polygon[0])/5, np.ceil(5*bounds_polygon[3])/5],
                                           [np.floor(5*bounds_polygon[0])/5, np.floor(5*bounds_polygon[1])/5],
                                           [np.ceil(5*bounds_polygon[2])/5, np.floor(5*bounds_polygon[1])/5],
                                           [np.ceil(5*bounds_polygon[2])/5, np.ceil(5*bounds_polygon[3])/5]])])

In [ ]:
def random_points_in_polygon(number, polygon):
    points = []
    min_x, min_y, max_x, max_y = polygon.bounds
    i= 0
    while i < number:
        point = Point(random.uniform(min_x, max_x), random.uniform(min_y, max_y))
        if polygon.contains(point):
            points.append(point)
            i += 1
    return gpd.GeoDataFrame(index=range(0,len(points)), 
                            crs={'init': 'epsg:4326'}, 
                            geometry=points) 

def plot_random_coordinate(gdf, size):
    return gdf.iloc[0:1].hvplot(geo=True, tiles='ESRI', alpha=0.5, width=500, height=500)*\
    gdf.iloc[1:2].hvplot(geo=True, alpha=1, width=500, height=500, color='red')*\
    random_points_in_polygon(size, gdf.iloc[0].geometry).hvplot(geo=True, color='yellow', 
                                                                title='Number of points: ' + str(size))


opts.defaults(
        opts.Overlay(active_tools=['wheel_zoom', 'pan']))

(plot_random_coordinate(gdf, 100) + plot_random_coordinate(gdf, 500) +\
plot_random_coordinate(gdf, 1000) + plot_random_coordinate(gdf, 5000)).cols(2)

## 2.2 Largeur de la grille désirée

In [ ]:
gdf_random_points = random_points_in_polygon(1000, gdf.iloc[0].geometry)

In [ ]:
gdf_random_points['x'] = np.round(gdf_random_points.geometry.x,1)
gdf_random_points['y'] = np.round(gdf_random_points.geometry.y,2)

In [ ]:
extent_polygon = list(gdf.iloc[2].geometry.bounds)
dist_x = np.round(abs(extent_polygon[0] - extent_polygon[2]),1)
dist_y = np.round(abs(extent_polygon[1] - extent_polygon[3]),1)

In [ ]:
import dask

sim_member = 1

   
row = gdf_random_points.iloc[sim_member-1]

da = ds['t2m'].sel(latitude=slice(0.01+np.round(row.y+dist_y/2,1), 
                                  -0.01+np.round(row.y-dist_y/2,1)),
                   longitude=slice(-0.01+np.round(row.x-dist_x/2,1), 
                                   0.01+np.round(row.x+dist_x/2,1)))

da_tmin = da.resample(time='1D').min()

da_tmin.coords['longitude'] = np.round(da_tmin.coords['longitude'] - 
                                       (da_tmin.coords['longitude'].min() - extent_polygon[0]),1)

da_tmin.coords['latitude'] = np.round(da_tmin.coords['latitude'] - 
                                      (da_tmin.coords['latitude'].min() - extent_polygon[1]),1)

da_tmin.expand_dims({'sim_member':sim_member}).to_netcdf(
    '/home/slanglois/Documents/ERA5/tmin_sim_{sim_member:0.0f}'.format(sim_member=sim_member))

In [ ]:
sim_member = 1



In [ ]:
output = []
for x in range(0,3):
    c = make_grid(sim_member=x,
                  gdf=gdf,
                  ds=ds)
    output.append(c)

In [ ]:
results = dask.compute(*output)

In [ ]:
extent_polygon

In [ ]:
da_tmin_reindexed = da_tmin.reindex({"latitude": new_index_latitude,
                 "longitude": new_index_longitude})

In [ ]:
da_tmin[10,:,:].hvplot()

In [ ]:
gdf.iloc[1:].hvplot(alpha=0.5, tiles='ESRI', geo=True, width = 600, height = 500)

In [ ]:
dist_y

In [ ]:
from IPython.display import display, Markdown

Markdown("""La grille pigée doit donc avoir un étendu de {}° sur la longitude et
   {}° sur la latitude""".format(dist_x, dist_y))